# **#1 Load dataset from GCP Cloud Storage**

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import storage

In [3]:
project_id = "onyx-sequencer-436913-i3"
client = storage.Client(project=project_id)

In [4]:
bucket_name = "cifake"
file_path = "cifake-real-and-ai-generated-synthetic-images.zip"
destination = "/content/cifake-real-and-ai-generated-synthetic-images.zip"

In [5]:
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(destination)

In [6]:
import zipfile

with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall("/content/cifake-real-and-ai-generated-synthetic-images")

# **#2 Prepare Training and Test Sets**

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, callbacks, optimizers
from tensorflow.keras.models import Sequential

In [8]:
train_data = '/content/cifake-real-and-ai-generated-synthetic-images/train'
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_data,
  labels='inferred',
  validation_split=0.7,
  subset="training",
  seed=123,
  image_size=(180, 180),
  batch_size=64)

Found 100000 files belonging to 2 classes.
Using 30000 files for training.


In [9]:
train_ds.class_names

['FAKE', 'REAL']

In [10]:
test_data = '/content/cifake-real-and-ai-generated-synthetic-images/test'
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data,
  labels='inferred',
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(180, 180),
  batch_size=64)

Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [11]:
test_ds.class_names

['FAKE', 'REAL']

In [12]:
X_train = []
y_train = []

for images, labels in train_ds:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train)

In [13]:
X_test = []
y_test = []

for images, labels in test_ds:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test)

In [14]:
X_train.shape

(30000, 180, 180, 3)

In [15]:
y_train.shape

(30000,)

In [16]:
X_test.shape

(6000, 180, 180, 3)

In [17]:
y_test.shape

(6000,)

In [18]:
np.unique(y_train)
np.unique(y_test)

array([0, 1], dtype=int32)

# **#3 Normalise X_train and X_test**

In [19]:
X_train = X_train/255
X_test = X_test/255

In [20]:
X_train

array([[[[0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         ...,
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ]],

        [[0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         ...,
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ]],

        [[0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         [0.42745098, 0.59607846, 0.72156864],
         ...,
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ],
         [0.48235294, 0.6156863 , 0.7254902 ]],

        ...,

        [[0.54901963, 0.5411765 , 0.48235294],
         [0.54901963, 0.5411765 , 0.48235294]

# **#4 CNN Baseline Model**

In [24]:
model = Sequential()

model.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(180, 180, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(16, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 178, 178, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 89, 89, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 87, 87, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 43, 43, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 29584)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │          59,170 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 61,941 (241.96 KB)

 Trainable params: 61,941 (241.96 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=30, batch_size=64)

Epoch 1/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8655 - loss: 0.3396
Epoch 2/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8743 - loss: 0.3218
Epoch 3/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8777 - loss: 0.3088
Epoch 4/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8875 - loss: 0.2891
Epoch 5/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8881 - loss: 0.2827
Epoch 6/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9002 - loss: 0.2609
Epoch 7/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9056 - loss: 0.2498
Epoch 8/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9069 - loss: 0.2401
Epoch 9/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9145 - loss: 0.2291
Epoch 10/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9081 - loss: 0.2286
Epoch 11/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9185 - loss: 0.2144
Epoch 12/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 

In [28]:
model.evaluate(X_test, y_test,verbose=0)

[0.42769983410835266, 0.8665000200271606]